In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

from dataset import DatasetConfig, DatasetBuilder
import os
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import defaultdict
from typing import Callable, Dict, List, Tuple
%matplotlib inline

In [2]:
ds_config =  DatasetConfig(model_dir='../dataset/pinecone/sparse/0/',
                           images_dir='../dataset/pinecone/images/',
                           batch_per_image=4,
                           is_training=True,
                           batch_size=256)

In [3]:
dataset = DatasetBuilder(ds_config)
ds = dataset.build()

In [4]:
lines = []
colors = []

def make_line(origin, direction, length=10):
    return origin, origin + direction * length

for elements in ds.take(100).as_numpy_iterator():
    origins = elements["origins"]
    directions = elements["directions"]
    pixels = elements["pixels"]
    N = origins.shape[0]
    for i in range(N):
        p0, p1 = make_line(origins[i, ...], directions[i, ...])
        lines.append(p0)
        lines.append(p1)
        colors.append(pixels[i, ...])
    print("Done")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [6]:
data = np.asarray(lines)
np.save('lines.npy', data)

colors = np.asarray(colors)
np.save('colors.npy', colors)

In [5]:
origins

array([[-3.0541387 ,  1.5242401 ,  0.7354028 ],
       [ 0.0336123 , -2.98742   , -0.9645931 ],
       [-1.8731095 ,  2.7394025 , -0.31046507],
       [-0.48280144, -3.125152  , -0.6705262 ],
       [-1.8731095 ,  2.7394025 , -0.31046507],
       [-1.8731095 ,  2.7394025 , -0.31046507],
       [ 0.8414927 , -3.5839822 ,  2.186288  ],
       [-1.8731095 ,  2.7394025 , -0.31046507],
       [-1.8731095 ,  2.7394025 , -0.31046507],
       [-3.0541387 ,  1.5242401 ,  0.7354028 ],
       [-1.8731095 ,  2.7394025 , -0.31046507],
       [-0.48280144, -3.125152  , -0.6705262 ],
       [-3.0541387 ,  1.5242401 ,  0.7354028 ],
       [-1.8731095 ,  2.7394025 , -0.31046507],
       [-0.48280144, -3.125152  , -0.6705262 ],
       [ 1.4567132 ,  0.5033875 , -2.3939292 ],
       [-3.0541387 ,  1.5242401 ,  0.7354028 ],
       [-1.8731095 ,  2.7394025 , -0.31046507],
       [-2.60579   ,  2.244224  , -0.3216559 ],
       [ 0.8414927 , -3.5839822 ,  2.186288  ],
       [ 1.4567132 ,  0.5033875 , -2.393